Neste kernel, mostrarei outras funções da API do BigQuery (BQ) úteis e forneçam recursos para você aprender sozinho.

Nosso primeiro objetivo será listar todas as tabelas disponíveis no conjunto de dados do Hacker News. Para começar, precisaremos carregar o cliente bigquery.

Você pode querer abrir a documentação enquanto segue:
- [Guia do cliente em Python] (https://googlecloudplatform.github.io/google-cloud-python/latest/bigquery/usage.html)
- [Referência da API do BQ] (https://googlecloudplatform.github.io/google-cloud-python/latest/bigquery/reference.html)

In [0]:
from google.cloud import bigquery

In [0]:
client = bigquery.Client()

Em seguida, carregaremos o conjunto de dados do Hacker News. Existem algumas dicas que você deve ter em mente aqui:

Para carregar um conjunto de dados, primeiro você precisa gerar uma referência de conjunto de dados para apontar BQ para ele.
Toda vez que você estiver trabalhando com o BQ do Kaggle, o nome do projeto será bigquery-public-data.
O Kaggle impõe algumas limitações à API do BQ para que ela funcione em nossa plataforma, mas a maioria dos comandos somente leitura funcionará. Uma exceção chave é client.list_datasets, então você pode querer olhar para um dos kernels iniciais para obter o ID do seu conjunto de dados.


In [0]:
hn_dataset_ref = client.dataset('hacker_news', project='bigquery-public-data')

O método client.dataset é chamado como se retornasse um conjunto de dados, mas na verdade nos fornece uma referência de conjunto de dados.

In [0]:
type(hn_dataset_ref)

Quando tivermos uma referência, podemos carregar o conjunto de dados real.


In [0]:
hn_dset = client.get_dataset(hn_dataset_ref)

In [0]:
type(hn_dset)

Agora podemos usar client.list_tables para obter informações sobre as tabelas dentro do conjunto de dados.


In [0]:
[x.table_id for x in client.list_tables(hn_dset)]

Vamos dar uma olhada no esquema da tabela `full`. Como nos conjuntos de dados, precisaremos passar uma referência à tabela para o método `client.get_table`.

In [0]:
hn_full = client.get_table(hn_dset.table('full'))

In [0]:
type(hn_full)

Em vez de olhar para a documentação, tentarei ser um pouco mais independente enquanto digito os comandos da tabela.

In [0]:
[command for command in dir(hn_full) if not command.startswith('_')]

O esquema parece útil, então vamos tentar isso.


In [0]:
hn_full.schema

Acontece que o esquema é uma entrada necessária para um dos comandos mais úteis no BQ: list_rows. List_rows retorna uma fatia de um conjunto de dados sem varrer qualquer outra seção da tabela. Se você já escreveu uma consulta BQ que incluísse uma cláusula de limite, que limitasse os dados retornados em vez dos dados verificados, você provavelmente desejaria, de fato, o list_rows.

Gostaria de ver um subconjunto das colunas, mas o parâmetro selected_fields requer um objeto de esquema como uma entrada. Precisamos primeiro criar um subconjunto do esquema para passar por esse parâmetro.

In [0]:
schema_subset = [col for col in hn_full.schema if col.name in ('by', 'title', 'time')]
results = [x for x in client.list_rows(hn_full, start_index=100, selected_fields=schema_subset, max_results=10)]

Você pode imprimir os resultados diretamente, mas acho que é um pouco difícil de ler. Ele exibe algum clichê, depois todos os valores e, em seguida, todas as chaves.


In [0]:
print(results)

Normalmente, eu provavelmente passaria isso em pandas antes de continuar o trabalho, mas por enquanto só podemos converter os resultados do google.cloud.bigquery.table.Row em dicts para obter uma versão que seja impressa um pouco mais bem.

In [0]:
for i in results:
    print(dict(i))

Suponha que nós quiséssemos verificar quais recursos teríamos consumido fazendo uma varredura completa da tabela ao invés de usar list_rows. Parece que o método num_bytes deve nos ajudar lá.

In [0]:
BYTES_PER_GB = 2**30
hn_full.num_bytes / BYTES_PER_GB

In [0]:
def estimate_gigabytes_scanned(query, bq_client):
    # see https://cloud.google.com/bigquery/docs/reference/rest/v2/jobs#configuration.dryRun
    my_job_config = bigquery.job.QueryJobConfig()
    my_job_config.dry_run = True
    my_job = bq_client.query(query, job_config=my_job_config)
    BYTES_PER_GB = 2**30
    return my_job.total_bytes_processed / BYTES_PER_GB

Agora podemos executar um teste rápido, verificando o impacto da seleção de uma coluna em relação a uma tabela inteira.

In [0]:
estimate_gigabytes_scanned("SELECT Id FROM `bigquery-public-data.hacker_news.stories`", client)

In [0]:
estimate_gigabytes_scanned("SELECT * FROM `bigquery-public-data.hacker_news.stories`", client)